In [8]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Options
from qiskit import QuantumCircuit, transpile, ClassicalRegister


import numpy as np
import math
import time
import json

#Insert token of your account
MY_TOKEN=''
service = QiskitRuntimeService(channel="ibm_quantum",url='https://auth.quantum-computing.ibm.com/api',token=MY_TOKEN)

In [9]:
def prep_meas(layout,backend=None):
    "
    """
    Parameters: 
     list: List of measured qubits
     backend: backend used for the quantum computation
        
    Returns: list of QuantumCircuit: Preparation of the 2**len(layout) bitstring configuration initial state,
                                     used to calibrate noisy measurement
    """
    
    nb_qubits=len(layout)
    qc_meas=list(QuantumCircuit(nb_qubits) for _ in range(2**nb_qubits))
    for k in range(2**nb_qubits):
        qc=QuantumCircuit(nb_qubits)#initialisation of the differents bit string configurations
        for p in range(nb_qubits):
            if (k//(2**p))%2==1:
                qc.x(p)
        qc.barrier()
        qc.measure_all()
        qc_meas[k]=transpile(qc,backend=backend,optimization_level=0,basis_gates=['cx','x','sx','rz','id'],initial_layout=layout)#measure all the qubits used 
    return qc_meas


In [6]:
backend='ibm_hanoi'#quantum machine used for the experiment
nr=100 #number of disorder realizations
nb_twirls_RC=100 #number of RC circuits per realization
lay=[1,4,7,10] #qubits used in the quantum simulation
nb_shots_meas=100000 #number of shots for the calibration of measurement
nb_shots_RC=1000 #number of shots per RC circuits


Backend = service.backend(backend)
#calibration circuit for measurement
qc_meas=prep_meas(lay,backend=Backend)



In [7]:
session_name='thermalization 1'

nb_qubits=len(lay)
options = Options(optimization_level=0,resilience_level=0)
ID=[]
with Session(service=service, backend=backend) as session:
    
    # Submit a request to the Sampler primitive within the session.
    sampler = Sampler(session=session, options=options)
    
    #launch circuits for measurement calibration
    job_meas = sampler.run(qc_meas, shots=nb_shots_meas)
    id_meas=job_meas.job_id()
    
    #launch RC circuits of each disorder realization
    for d in range(1,nr+1):
        qc_disorder=list(QuantumCircuit.from_qasm_file('data/disorder_realization_'+str(d)+'/T=10/RC/recompiled_circuit_disorder_'+str(d)+'_T=10_RC_'+str(k)+'.qasm') for k in range(nb_twirls_RC))
        job_disorder = sampler.run(qc_disorder, shots=nb_shots_RC)
        ID.append(job_disorder.job_id())
    
    with open('data/QC/session '+str(session_name)+'/ID_meas_backend='+str(backend)+'.json',"w") as json_file:
        json.dump(id_meas, json_file)
        
    with open('data/QC/session '+str(session_name)+'/ID_RC_backend='+str(backend)+'.json',"w") as json_file:
        json.dump(ID, json_file)
    
    job_meas=service.job(id_meas)
    result_meas = job_meas.result().quasi_dists

    #compute the calibration matrix of noisy measurements
    meas_matrix=list(list(0 for _ in range(2**nb_qubits)) for _ in range(2**nb_qubits))
    for k in range(2**nb_qubits):
        for key in result_meas[k].keys():
            meas_matrix[int(key)][k]=result_meas[k][key]
    
    #save the result
    with open('data/QC/session '+str(session_name)+'/measurement_backend='+str(backend)+'.json',"w") as json_file:
        json.dump(meas_matrix, json_file)

    #save result from QC for each disorder realization
    for d in range(nr):
        job_disorder=service.job(ID[d])
        result_disorder = job_disorder.result().quasi_dists
        with open('data/QC/session '+str(session_name)+'/result_disorder_realization_'+str(d+1)+'_backend='+str(backend)+'.json',"w") as json_file:
            json.dump(result_disorder, json_file)

    